<a href="https://colab.research.google.com/github/vladgap/Various/blob/main/MLN-281121_Regres_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import numpy as np
# import pandas as pd
from sklearn import preprocessing

# StandardScaler

In [50]:
x=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
# x=np.array(x)
scaler = preprocessing.StandardScaler().fit(x)
print(scaler.mean_, scaler.scale_)
x_scaled = scaler.transform(x)
print (x_scaled)
x_back=scaler.inverse_transform(x_scaled)
print (x_back)
scaler.inverse_transform([[0,0,0]])
# help(scaler)

[ 0.          0.33333333 -0.16666667] [1.15470054 0.74535599 0.68718427]
[[ 1.73205081  0.89442719  0.24253563]
 [-0.8660254  -0.4472136   1.69774938]
 [ 0.         -0.4472136   0.24253563]
 [ 0.8660254   0.89442719  0.24253563]
 [-0.8660254  -1.78885438 -1.21267813]
 [-0.8660254   0.89442719 -1.21267813]]
[[ 2.  1.  0.]
 [-1.  0.  1.]
 [ 0.  0.  0.]
 [ 1.  1.  0.]
 [-1. -1. -1.]
 [-1.  1. -1.]]


array([[ 0.        ,  0.33333333, -0.16666667]])

# Single Neuron

In [134]:
class Neuron:
    """A single neuron with an activation function.
       Attributes:
          inputs: The number of inputs in the perceptron, not counting the bias.
          bias:   The bias term. By defaul it's 1.0.
          activ:  The activation function: linear (default), relu, sigmoid."""

    def __init__(self, inputs, bias = 1.0, activ = 'linear'):
        """Return a new Perceptron object with the specified number of inputs (+1 for the bias) and random initial weights.""" 
        self.weights = (np.random.rand(inputs+1) * 2) - 1 
        self.bias = bias
        self.activ = activ

    def run(self, x):
        """Run the perceptron. x is a python list with the input values."""
        sum = np.dot(np.append(x,self.bias),self.weights)
        if self.activ == 'linear':
          return sum
        if self.activ == 'sigmoid':
          return self.sigmoid(sum)
        if self.activ == 'relu':
          return self.relu(sum)
        if self.activ == 'mrelu':
          return self.mrelu(sum)

    def set_weights(self, w_init):
        """Overrides the np.random.rand() weights and the bias weight."""
        # w_init is a list of floats. Organize it as you'd like.
        self.weights=np.array(w_init)

    def set_activ(self, activ, param=0):
        """Overrides the 'linear' activation function."""
        self.activ = activ
        self.param = param

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))
    
    def relu(self, x):
        # return the output of the relu function applied to x
        if x >= 0:
          return x
        return 0

    def mrelu(self, x):
        # return the output of the modified relu function applied to x
        if x >= 0:
          return x
        return self.param*x

In [52]:
4>=4

True

In [139]:
neu=Neuron(inputs=2)
neu.set_weights([10,10,-15]) 
neu.set_activ('mrelu', param=0.01)
neu.run([1,0])

-0.05

## AND gate

In [54]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-15]) #AND gate

print("AND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


AND Gate:
0 0 = 0.0000003059
0 1 = 0.0066928509
1 0 = 0.0066928509
1 1 = 0.9933071491


## OR gate

In [55]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-5]) #OR gate

print("OR Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


OR Gate:
0 0 = 0.0066928509
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.9999996941


## NAND gate

In [56]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([-10,-10,15]) #NAND gate

print("NAND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))

NAND Gate:
0 0 = 0.9999996941
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.0066928509


# Multilayer neuron

In [154]:
class MultiLayerNeuron:     
    """A multilayer neuron class that uses the Neuron class above.
       Builds a list of neurons with the specific activation function.
       The activation function may be modified later using the set_activ method.
       For example: mln.network[layer][neuron].set_activ('linear').
       Attributes:
          layers:  A list with the number of neurons per layer. Including the input (first) and the output (last) layers.
          bias:    The bias term. The same bias is used for all neurons.
          eta:     The learning rate.
          activ:   The activation function: linear (default), relu, sigmoid."""

    def __init__(self, layers, bias = 1.0, eta = 0.5, activ='linear'):
        """Return a new MLP object with the specified parameters.""" 
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.eta = eta
        self.network = [] # The list of lists of neurons (perceptrons).
        self.values = []  # The list of lists of neurons' (perceptrons') output values.
        self.d = []       # The list of lists of error terms (lowercase deltas)
        self.activ = activ

        # 2 nested loops to create neurons layer by layer
        for i in range(len(self.layers)): # outer loop iterates on each layer
            self.values.append([]) #The new list of values will be filled with zeros, for every neuron in the layer. 
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d.append([])
            self.d[i] = [0.0 for j in range(self.layers[i])]                        
            self.network.append([])
            if i > 0:      #network[0] is the input layer, so it has no neurons
                for j in range(self.layers[i]): # inner loop iterates on each neuron in a layer
                    neur=Neuron(inputs = self.layers[i-1], bias = self.bias, activ = self.activ) # 
                    self.network[i].append(neur) # adding j perceptrons
        
        self.network = np.array([np.array(x) for x in self.network],dtype=object) #transforms list of lists to numpy array
        self.values = np.array([np.array(x) for x in self.values],dtype=object)
        self.d = np.array([np.array(x) for x in self.d],dtype=object)

    def set_weights(self, w_init): # set_weights of the MultiLayer class
        """Set the weights. 
           w_init is a list of lists with the weights for all but the input layer."""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j]) # set_weights for each perceptron i

    def set_activ(self, activ, param=0):
        """Set the activation function to every neurons."""
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                self.network[i][j].set_activ(activ, param) # set_activ for each neuron
        self.param=param
    
    def set_output_activ(self, activ, param=0):
        """Set the activation function to the last (output) neurons."""
        i = len(self.network)-1
        for j in range(self.layers[i]):
            self.network[i][j].set_activ(activ, param) 

    def printWeights(self):
        """Displays a summary of weights and activation functions per layer and neuron."""
        print()
        print('Layer 0 is the Input Layer')
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i,"Neuron",j,":",self.network[i][j].weights,self.network[i][j].activ)
        print()

    def run(self, x):
        """Feed a sample x into the MultiLayer Neuron."""
        x = np.array(x,dtype=object)
        self.values[0] = x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):  
                self.values[i][j] = self.network[i][j].run(self.values[i-1]) #runs preceptrons with the previous outputs
        return self.values[-1]

    def bp_classif(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative of the sigmoid function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        self.d[-1] = outputs * (1 - outputs) * (error) # derivative of the SIGMOID function 
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = self.values[i][h] * (1-self.values[i][h]) * fwd_error # derivative of the SIGMOID function
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    if k==self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] # applying the delta rule
                    self.network[i][j].weights[k] += delta
        return MSE

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))

    def deriv(self, value, i, j=0):
        if self.network[i][j].activ == 'linear':
          # print ('lin')
          return 1
        if self.network[i][j].activ == 'sigmoid':
          # print ('sig')
          return self.sigmoid(value)*(1-self.sigmoid(value))
        if self.network[i][j].activ == 'relu':
          if value > 0:
            # print ('re>')
            return 1
          else:
            # print ('re<')
            return 0
        if self.network[i][j].activ == 'mrelu':
          if value > 0:
            return 1
          else:
            return self.param

    def bp_regres(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative according each neuron's activation function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        for j in range (len(outputs)):
            self.d[-1][j] = self.deriv(outputs[j], len(self.network)-1) * error
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k] 
                self.d[i][h] = self.deriv(self.values[i][h], i, h) * fwd_error
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    # output=sum(weight*value)+bias*bias_weight
                    if k==self.layers[i-1]:
                        # ∂output/∂bias_weight=bias
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        # ∂output/∂weight=value
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] 
                    self.network[i][j].weights[k] += delta # applying the delta rule
        return MSE

## XOR gate=(OR+NAND)+AND

In [58]:
#test code
mln1 = MultiLayerNeuron(layers=[2,2,1])  #mln1
mln1.set_weights([[[-10,-10,15],[15,15,-10]],[[10,10,-15]]])
mln1.set_activ('sigmoid') #linear is by default

mln1.printWeights()
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln1.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln1.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln1.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln1.run([1,1])[0]))


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-10 -10  15] sigmoid
Layer 1 Neuron 1 : [ 15  15 -10] sigmoid
Layer 2 Neuron 0 : [ 10  10 -15] sigmoid

XOR Gate:
0 0 = 0.0066958493
0 1 = 0.9923558642
1 0 = 0.9923558642
1 1 = 0.0071528098


## Training 

In [59]:
mln2 = MultiLayerNeuron(layers=[2,2,1])
mln2.set_activ('sigmoid') #linear is by default
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(2000):
    MSE = 0.0
    MSE += mln2.bp_classif([0,0],[0])
    MSE += mln2.bp_classif([0,1],[1])
    MSE += mln2.bp_classif([1,0],[1])
    MSE += mln2.bp_classif([1,1],[0])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln2.printWeights()
    
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln2.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln2.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln2.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln2.run([1,1])[0]))


Training Neural Network as an XOR Gate...

1.1034003191636055
1.0693832090570776
0.8677396198411216
0.6312101578380522
0.06632064300724003
0.0228574228664142
0.013176348921201316
0.009117910172248831
0.006921752807609517
0.005556001943169528

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-6.14667767 -6.14271178  2.41430799] sigmoid
Layer 1 Neuron 1 : [-4.30869095 -4.30444496  6.37552695] sigmoid
Layer 2 Neuron 0 : [-8.75625995  8.55002672 -3.98658864] sigmoid

XOR Gate:
0 0 = 0.0296391912
0 1 = 0.9677316470
1 0 = 0.9676434539
1 1 = 0.0405971126


# Regres

In [185]:
class Regres:
    def __init__(self, layers, bias=1.0, eta=0.5, ):
        self.layers = layers
        self.bias = bias
        self.eta = eta
        self.regres_network = MultiLayerNeuron(layers=layers, bias=bias, eta=eta)

    def set_weights(self, w_init):
        self.regres_network.set_weights(w_init)

    def set_hidden_activ(self, activ, param=0):
        self.regres_network.set_activ(activ, param=0)
        self.regres_network.set_output_activ('linear')

    def fit(self, X, y, epochs):
        self.epochs=epochs
        self.weight_history=[]
        self.MSE_history=[]
        for i in range(self.epochs):
            weight_epoch=[]
            MSE = 0.0
            for j in range (len(X)):
                MSE +=  self.regres_network.bp_regres(X[j],[y[j]])
            MSE = MSE / len(X)
            for m in range(1,len(self.regres_network.network)):
                for n in range(self.layers[m]):
                    weight_epoch.append(self.regres_network.network[m][n].weights)
            self.weight_history.append(weight_epoch)
            self.MSE_history.append(MSE)

    def printWeights(self):
        self.regres_network.printWeights()

    def run(self, x):
        return self.regres_network.run(x)

    def predict(self, x):
        y=[]
        for i in range(len(x)):
            y.append(self.run(x[i])[0])
        return y

# Regres_scale

In [61]:
# nuzhno podumat' kak vesa obratno perevesti. mozhet inverse_transform
class Regres_scale:
    def __init__(self, layers, bias=1.0, eta=0.5, ):
        self.layers = layers
        self.bias = bias
        self.eta = eta
        # self.epochs = epochs      
        self.regres_network = MultiLayerNeuron(layers=layers, bias=bias, eta=eta)

    def set_weights(self, w_init):
        self.regres_network.set_weights(w_init)

    def set_hidden_activ(self, activ):
        self.regres_network.set_activ(activ)
        self.regres_network.set_output_activ('linear')

    def fit(self, x, y, epochs):
        self.scaler = preprocessing.StandardScaler().fit(x)
        x_scaled=self.scaler.transform(x)
        self.epochs=epochs
        for i in range(epochs):
            MSE = 0.0
            for j in range (len(x)):
                MSE +=  self.regres_network.bp_regres(x_scaled[j],[y[j]])
            MSE = MSE / len(x)
            # dobavit' istoriyu kak v Regres

    def printWeights(self):
        self.regres_network.printWeights()

    def run(self, x):
        return self.regres_network.run(x)

    def predict(self, x):
        y=[]
        for i in range(len(x)):
            y.append(self.run(x[i])[0])
        return y

    def get_standard_params(self, x):
        x2=np.array(x)
        mean=x2.mean(axis=0)
        stdev=x2.std(axis=0)
        return mean, stdev

# Example1 -- weights

In [62]:
print ("1 1 =",mln1.run([1,1]))
print (mln1.network[1][0].weights) # network is list of lists of perceptrons. Each has attribute "weights"

1 1 = [0.00715281]
[-10 -10  15]


In [63]:
mln1=MultiLayerNeuron(layers=[2,2,1])
mln1.set_activ('relu')
mln1.printWeights()

mln1.set_output_activ('sigmoid') # setting the output activ func
mln1.printWeights()

mln1.network[1][0].set_activ('linear') # changing specific activ func
mln1.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.89920023 0.22580934 0.01236954] relu
Layer 1 Neuron 1 : [ 0.56378196 -0.31550563  0.9658366 ] relu
Layer 2 Neuron 0 : [-0.16169181  0.87379262  0.39717317] relu


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.89920023 0.22580934 0.01236954] relu
Layer 1 Neuron 1 : [ 0.56378196 -0.31550563  0.9658366 ] relu
Layer 2 Neuron 0 : [-0.16169181  0.87379262  0.39717317] sigmoid


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.89920023 0.22580934 0.01236954] linear
Layer 1 Neuron 1 : [ 0.56378196 -0.31550563  0.9658366 ] relu
Layer 2 Neuron 0 : [-0.16169181  0.87379262  0.39717317] sigmoid



# Example 2 -- sigmoid, bp_classif, XOR

In [64]:
mln2 = MultiLayerNeuron(layers=[2,2,1], eta=0.5)
print (mln2.network[1][1].weights)
mln2.set_activ('sigmoid') #linear is by default
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(2000):
    MSE = 0.0
    MSE += mln2.bp_classif([0,0],[0])
    MSE += mln2.bp_classif([0,1],[1])
    MSE += mln2.bp_classif([1,0],[1])
    MSE += mln2.bp_classif([1,1],[0])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln2.printWeights()
    
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln2.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln2.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln2.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln2.run([1,1])[0]))

[ 0.65526687  0.52866591 -0.32701178]

Training Neural Network as an XOR Gate...

1.1704325926091503
1.076266158174247
1.043886583872709
0.5103941749660728
0.049110267984580236
0.02034966571699586
0.012359226323665144
0.008752454513068934
0.006728961356711569
0.005443750426326519

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-5.84782156 -5.85443513  2.22132496] sigmoid
Layer 1 Neuron 1 : [ 4.40035021  4.39831416 -6.84625758] sigmoid
Layer 2 Neuron 0 : [-8.50509845 -8.64412132  4.21891094] sigmoid

XOR Gate:
0 0 = 0.0303796643
0 1 = 0.9648348281
1 0 = 0.9647428988
1 1 = 0.0338470156


# Example 3 -- linear activs, bp_regres

In [98]:
mln3 = MultiLayerNeuron(layers=[3,1], eta=0.3)
# mln3.set_weights([[[5.,-0.9,2.,0.]]])
mln3.set_weights([[[0.,0.,0.,0.]]])
mln3.printWeights()
print("\nTraining Neural Network...\n")
for i in range(400):
    MSE = 0.0
    MSE += mln3.bp_regres([0.7759,	0.1104,	0.9977,],5.764286995)
    MSE += mln3.bp_regres([0.9692,	0.6961,	0.8483,],5.84646758)
    MSE += mln3.bp_regres([0.0265,	0.399,	0.5375,],0.808633075)
    MSE += mln3.bp_regres([0.7694,	0.5051,	0.2542,],3.850298589)
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln3.printWeights()




Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0. 0. 0. 0.] linear


Training Neural Network...

54.560573414122615
9.652321993077698e-13

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 4.99973369e+00 -9.99625006e-01  1.99991681e+00  3.52253294e-05] linear



In [111]:
mln3 = MultiLayerNeuron(layers=[2,1], eta=0.1)
# mln3.set_weights([[[2.,1.]]])
# mln3.set_weights([[[0.,0.,0.,0.]]])
mln3.printWeights()
print("\nTraining Neural Network...\n")
for i in range(400):
    MSE = 0.0
    MSE += mln3.bp_regres([0.,0.],0.)
    MSE += mln3.bp_regres([1.,1.],3.)
    MSE += mln3.bp_regres([2.,0.],4.)
    MSE += mln3.bp_regres([3.,1.],7.)
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln3.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.68465174 0.99089057 0.61023135] linear


Training Neural Network...

3.875109727823518
9.555017328664439e-46

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [2.00000000e+00 1.00000000e+00 1.28266469e-42] linear



In [112]:
mln3 = MultiLayerNeuron(layers=[1,1], eta=0.1)
# mln3.set_weights([[[2.,1.]]])
# mln3.set_weights([[[0.,0.,0.,0.]]])
mln3.printWeights()
print("\nTraining Neural Network...\n")
for i in range(400):
    MSE = 0.0
    MSE += mln3.bp_regres([0.],0.)
    MSE += mln3.bp_regres([1.],2.)
    MSE += mln3.bp_regres([2.],4.)
    MSE += mln3.bp_regres([3.],6.)
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln3.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.55949513  0.86399944] linear


Training Neural Network...

15.916998042547032
9.293510831493277e-49

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [2.00000000e+00 4.00025156e-44] linear



# Example 4 -- sigmoid, bp_regres

In [88]:
# generating output values
mln4 = MultiLayerNeuron(layers=[3,2,1], eta=0.3)
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[-1,2,3]]])
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1], eta=0.3)
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')

for i in range(1000):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0  1 -2  1] sigmoid
Layer 1 Neuron 1 : [ 1 -2  0  1] sigmoid
Layer 2 Neuron 0 : [-1  2  3] linear



'original values:'

[3.5813200792821274,
 3.731058578630005,
 3.731058578630005,
 3.119202922022118,
 3.880797077977882,
 2.2563920540063265]


MSE= 16.716674234626698
MSE= 0.6310297986858845
MSE= 1.478703670208817
MSE= 2.103572521455943
MSE= 1.9332067650968952

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 251.10660263 -501.13392304  252.97682973   -6.20030076] sigmoid
Layer 1 Neuron 1 : [ 475.24547825 -947.61015604  477.7836324    -6.74983559] sigmoid
Layer 2 Neuron 0 : [2.95371798 7.9291997  2.79979996] linear



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: RuntimeWarning: overflow encountered in exp


'predicted values:'

[2.979492921129191,
 2.953990563729689,
 2.8150551953706873,
 2.7997999619712557,
 2.800154883354388,
 2.7997999619712557]

# Example 5 -- various, bp_regres

In [67]:
mln4 = MultiLayerNeuron(layers=[3,2,1,1]) # generating MLN with random weights
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[1,0,1]],[[-1,2]]]) # setting weights
mln4.printWeights()

y4=[] # generating output values
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1,1], eta=0.3) # generating MLN with random weights
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')

for i in range(4000):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE', i, '=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0  1 -2  1] relu
Layer 1 Neuron 1 : [ 1 -2  0  1] relu
Layer 2 Neuron 0 : [1 0 1] sigmoid
Layer 3 Neuron 0 : [-1  2] linear



'original values:'

[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]


MSE 0 = 2.652662917230955
MSE 200 = 0.04520465482612365
MSE 400 = 0.04118409816905768
MSE 600 = 0.04024060735659487
MSE 800 = 0.039826959023904755
MSE 1000 = 0.03967275092502534
MSE 1200 = 0.0395450916538607
MSE 1400 = 0.03944010017907626
MSE 1600 = 0.03935402611072759
MSE 1800 = 0.03928325163598307
MSE 2000 = 0.039224683730955875
MSE 2200 = 0.0391758368127386
MSE 2400 = 0.039134764035660595
MSE 2600 = 0.03909995179388238
MSE 2800 = 0.03907022156100762
MSE 3000 = 0.039044650481688224
MSE 3200 = 0.03902251047217594
MSE 3400 = 0.039003222443119184
MSE 3600 = 0.03898632205735386
MSE 3800 = 0.03897143405383046

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.39221434  0.40762168 -0.57138933 -1.19245729] relu
Layer 1 Neuron 1 : [-0.97153621  0.35941008 -0.99323592 -0.44718933] relu
Layer 2 Neuron 0 : [-0.209094    1.16140419 -2.36877473] sigmoid
Layer 3 Neuron 0 : [-0.21628877  1.10467613] linear



'predicted values:'

[1.0861650590568244,
 1.0861650590568244,
 1.0861650590568244,
 1.0861650590568244,
 1.0475080834426886,
 1.0067118223095244]

# Example 7 -- Regres class

In [68]:
reg1=Regres(layers=[3,2,1,1], eta=0.3)
reg1.set_weights([[[1.,1.,0.,0.],[0.,0.,0.,0.]],[[0.,0.,0.]],[[0.,0.]]])
X=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
y=[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]
reg1.set_hidden_activ('relu')
reg1.regres_network.network[2][0].set_activ('sigmoid') # changing specific activ func
reg1.fit(X,y, epochs=10)
reg1.printWeights()
pred=reg1.predict(X)
print (pred)
print()
# display ('weight_history',reg1.weight_history)
display ('MSE',reg1.MSE_history)


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 9.99023284e-01  1.00152136e+00 -9.19439041e-04  2.95824808e-03] relu
Layer 1 Neuron 1 : [0. 0. 0. 0.] relu
Layer 2 Neuron 0 : [-0.0528578   0.         -0.00270726] sigmoid
Layer 3 Neuron 0 : [0.40569153 0.81590977] linear

[1.0024198942493239, 1.01848095600904, 1.0184650968999405, 1.0077510108992154, 1.01848095600904, 1.018446775675626]



'MSE'

[0.8936248829442258,
 0.04857883327236249,
 0.0481469881158581,
 0.04777680393002976,
 0.04744442311981101,
 0.047145341800303116,
 0.04687567299207188,
 0.04663204485218531,
 0.04641152086265987,
 0.046211534780671255]

In [69]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



In [70]:
# nuzhno podumat' kak vesa obratno perevesti. mozhet inverse_transform
reg2=Regres_scale(layers=[3,2,1,1], eta=0.3)
reg2.set_weights([[[1.,1.,0.,0.],[0.,0.,0.,0.]],[[0.,0.,0.]],[[0.,0.]]])
X=[
[2,1,0],
[-1,0,1],
[0,0,0],
[1,1,0],
[-1,-1,-1],
[-1,1,-1],
]
y=[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]
reg2.set_hidden_activ('relu')
reg2.regres_network.network[2][0].set_activ('sigmoid') # changing specific activ func
reg2.fit(X,y,epochs=200)
reg2.printWeights()
pred=reg2.predict(X)
pred


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.90094016  1.08501405 -0.07498197  0.09504859] relu
Layer 1 Neuron 1 : [0. 0. 0. 0.] relu
Layer 2 Neuron 0 : [-0.17097451  0.         -0.24461117] sigmoid
Layer 3 Neuron 0 : [0.34647447 0.87394367] linear



[0.9847645694327527,
 1.0260980418625074,
 1.0247126609442043,
 0.9966822456678671,
 1.0260980418625074,
 1.0209521040416654]

# Example 6 -- https://mmuratarat.github.io/2020-01-09/backpropagation

In [92]:
# generating output values
mln6 = MultiLayerNeuron(layers=[5,3,1], eta=.5)
mln6.set_activ('sigmoid')
mln6.set_output_activ('linear')
mln6.set_weights([[[.19,.33,.4,.51,.54,.1],
                   [.55,.16,.35,.85,.49,.1],
                   [.76,.97,.7,.85,.57,.1]],
                  [[.1,.03,-.17,.1]]
                  ]) # setting initial weights
mln6.printWeights()

# training
for i in range(100):
    MSE = 0.0
    MSE += mln6.bp_regres([.5,.1,1,0,0],[0.1])
    MSE += mln6.bp_regres([.3,.2,0,1,0],[.6])
    MSE += mln6.bp_regres([.7,.9,0,0,1],[.4])
    MSE += mln6.bp_regres([.8,.1,1,0,0],[.1])
    MSE = MSE / 4
    if(i%10 == 0):
        print ('MSE', i, '=',MSE)

mln6.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.19 0.33 0.4  0.51 0.54 0.1 ] sigmoid
Layer 1 Neuron 1 : [0.55 0.16 0.35 0.85 0.49 0.1 ] sigmoid
Layer 1 Neuron 2 : [0.76 0.97 0.7  0.85 0.57 0.1 ] sigmoid
Layer 2 Neuron 0 : [ 0.1   0.03 -0.17  0.1 ] linear

MSE 0 = 3.097759865123268
MSE 10 = 0.2948017967667316
MSE 20 = 0.11235016703947856
MSE 30 = 0.039031749833149484
MSE 40 = 0.005346702764168406
MSE 50 = 0.00047155150910413527
MSE 60 = 9.822149422722454e-05
MSE 70 = 6.0504726410544755e-05
MSE 80 = 4.67746394940529e-05
MSE 90 = 3.966114337742912e-05

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.92948745 -0.3432972  -0.4785581   0.21811547 -0.0456272  -1.65606983] sigmoid
Layer 1 Neuron 1 : [-1.01311017 -0.23274925 -1.57166574  1.18384536  0.1929425  -1.78487787] sigmoid
Layer 1 Neuron 2 : [-0.09277171 -0.3986799   1.19699659 -0.59778185 -0.68424799 -2.10503325] sigmoid
Layer 2 Neuron 0 : [ 0.25752661  0.97942477 -1.06640139  0.35525422] linear



# Example 8 -- Relu check

In [127]:
reg2=Regres(layers=[1,1], eta=0.1)
X=[[0.],[1.],[2.],[3.],[4.],[5.]]
y=[0.,1.,2.,3.,4.,5.]
reg2.printWeights()

reg2.set_hidden_activ('linear')
reg2.fit(X,y,epochs=5)
reg2.printWeights()
pred=reg2.predict(X)
print (pred)
print()
display ('weight_history',reg2.weight_history)
display ('MSE',reg2.MSE_history)


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.98341389 -0.65431364] linear


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [1.00929573 0.01642246] linear

[0.01642246285429991, 1.025718196545412, 2.035013930236524, 3.044309663927636, 4.053605397618749, 5.062901131309861]



'weight_history'

[[array([1.00929573, 0.01642246])],
 [array([1.00929573, 0.01642246])],
 [array([1.00929573, 0.01642246])],
 [array([1.00929573, 0.01642246])],
 [array([1.00929573, 0.01642246])]]

'MSE'

[10.791498637420688,
 1.5199521179822308,
 0.2277264205291422,
 0.03411905019472734,
 0.005111877591916453]

# Example 9 -- mrelu check. working

In [181]:
mln4 = MultiLayerNeuron(layers=[1,2,1]) # generating MLN with random weights
mln4.set_activ('mrelu', param=0.1)
# mln4.set_activ('relu')
mln4.set_output_activ('linear')
mln4.set_weights([[[-1.1,1.74],[3.35,-6.44]],[[-1.41,2.05,2.2]]]) # setting weights
mln4.printWeights()

y4=[] # generating output values
y4.append(mln4.run([0])[0])
y4.append(mln4.run([1])[0])
y4.append(mln4.run([2])[0])
y4.append(mln4.run([3])[0])
y4.append(mln4.run([4])[0])
y4.append(mln4.run([5])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[1,2,1], eta=0.005) # generating MLN with random weights
mln4.set_activ('mrelu', param=0.1)
# mln4.set_activ('relu')
mln4.set_output_activ('linear')

for i in range(4000):
    MSE = 0.0
    MSE += mln4.bp_regres([0],y4[0])
    MSE += mln4.bp_regres([1],y4[1])
    MSE += mln4.bp_regres([2],y4[2])
    MSE += mln4.bp_regres([3],y4[3])
    MSE += mln4.bp_regres([4],y4[4])
    MSE += mln4.bp_regres([5],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE', i, '=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([0])[0])
y4.append(mln4.run([1])[0])
y4.append(mln4.run([2])[0])
y4.append(mln4.run([3])[0])
y4.append(mln4.run([4])[0])
y4.append(mln4.run([5])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-1.1   1.74] mrelu
Layer 1 Neuron 1 : [ 3.35 -6.44] mrelu
Layer 2 Neuron 0 : [-1.41  2.05  2.2 ] linear



'original values:'

[-1.5735999999999999,
 0.6641500000000005,
 2.7978599999999996,
 9.82046,
 16.843059999999998,
 23.865659999999995]


MSE 0 = 212.9818742647429
MSE 200 = 1.95241717702793
MSE 400 = 0.5487042459412064
MSE 600 = 0.1897231055460981
MSE 800 = 0.0932086194229352
MSE 1000 = 0.05482719615365068
MSE 1200 = 0.03288718988441663
MSE 1400 = 0.02071105022734419
MSE 1600 = 0.014510680382621393
MSE 1800 = 0.010049610090113318
MSE 2000 = 4.648542363502512
MSE 2200 = 9.412987763956323
MSE 2400 = 0.08257332918805783
MSE 2600 = 0.019762614060483637
MSE 2800 = 0.008819827891358297
MSE 3000 = 0.004476111666594595
MSE 3200 = 0.0023630850374690717
MSE 3400 = 0.0012926690073169931
MSE 3600 = 0.001291660220896314
MSE 3800 = 0.0012916379057849548

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [1.46716344e+00 5.68918381e-05] mrelu
Layer 1 Neuron 1 : [ 3.05458588 -6.11869068] mrelu
Layer 2 Neuron 0 : [ 1.12235938  1.75994936 -0.47882522] linear



'predicted values:'

[-1.5556199395472112,
 0.62865636406093,
 2.812932667669071,
 9.820456263860041,
 16.843057386158755,
 23.865658508457464]

In [254]:
reg2=Regres(layers=[1,2,1], eta=0.0001)
X=[[1],[2],[3],[4],[5]]
y=[1.,4.,9.,16.,25.]
# reg2.printWeights()

reg2.set_hidden_activ('mrelu', param=0.1)
reg2.fit(X,y,epochs=500)
reg2.printWeights()
pred=reg2.predict(X)
print (pred)
print()
# display ('weight_history',reg2.weight_history)
display ('MSE',np.array(reg2.MSE_history).min())


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.32341138  0.76459193] mrelu
Layer 1 Neuron 1 : [-0.25507667  0.04429681] mrelu
Layer 2 Neuron 0 : [-0.21562986  0.62444639  4.34405303] linear

[4.248921328482143, 4.318658478632218, 4.344053029757812, 4.344053029757812, 4.344053029757812]



'MSE'

476.1662302367531